In [6]:
import os
import shutil
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
# Part 2: Create Smaller Dataset
def create_smaller_dataset(source_dir, dest_dir, num_images_per_class=250):
    if not os.path.exists(dest_dir):
        os.makedirs(dest_dir)

    for class_dir in os.listdir(source_dir):
        class_path = os.path.join(source_dir, class_dir)
        dest_class_path = os.path.join(dest_dir, class_dir)
        
        if not os.path.exists(dest_class_path):
            os.makedirs(dest_class_path)
        
        images = os.listdir(class_path)
        random.shuffle(images)
        selected_images = images[:num_images_per_class]
        
        for img in selected_images:
            src_img_path = os.path.join(class_path, img)
            dest_img_path = os.path.join(dest_class_path, img)
            shutil.copy(src_img_path, dest_img_path)

# Load treatments from text files
def load_treatment(stage):
    file_path = os.path.join(r"C:\Users\tanis\OneDrive - UPES\datasets\preditive analysis lab\treatment_files", f"{stage}.txt")
    with open(file_path, "r") as file:
        return file.read()

# Prepare data from the smaller dataset
def prepare_data(image_directory):
    # Example using ImageDataGenerator for loading images
    datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # Normalize and split data

    train_generator = datagen.flow_from_directory(
        image_directory,
        target_size=(64, 64),  # Reduced image size for faster training
        batch_size=64,  # Increased batch size
        class_mode='categorical',
        subset='training'
    )
    
    validation_generator = datagen.flow_from_directory(
        image_directory,
        target_size=(64, 64),  # Reduced image size for faster training
        batch_size=64,  # Increased batch size
        class_mode='categorical',
        subset='validation'
    )
    
    return train_generator, validation_generator

# Main setup
def setup():
    # Set source and destination directories
    source_directory = r"C:\Users\tanis\OneDrive - UPES\datasets\preditive analysis lab\Data"
    destination_directory = r"C:\Users\tanis\OneDrive - UPES\datasets\preditive analysis lab\Data_small"
    
    # Create a smaller dataset with 250 images per class (total of 1000 images)
    create_smaller_dataset(source_directory, destination_directory, num_images_per_class=250)

    # Prepare the reduced data for training
    image_directory = destination_directory
    train_data, val_data = prepare_data(image_directory)

    # Create CNN model
    input_shape = (64, 64, 3)  # Reduced image size
    model = create_cnn_model(input_shape)
    
    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model, train_data, val_data

def create_cnn_model(input_shape):
    model = models.Sequential()
    
    # Convolutional layers
    model.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape))  # Reduced filters
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(32, (3, 3), activation='relu'))
    
    # Flatten and Dense layers
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(4, activation='softmax'))  # 4 classes: non-demented, very mild, mild, moderate
    
    return model

# Part 3: Train the Model
def train_model(model, train_data, val_data):
    # Train the model on the smaller dataset
    model.fit(train_data, epochs=10, validation_data=val_data)
    return model

# Call setup and train
if __name__ == "__main__":
    model, train_data, val_data = setup()
    trained_model = train_model(model, train_data, val_data)

Found 6765 images belonging to 4 classes.
Found 1690 images belonging to 4 classes.
Epoch 1/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 34s 277ms/step - accuracy: 0.4381 - loss: 1.1676 - val_accuracy: 0.4817 - val_loss: 1.2911
Epoch 2/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 20s 179ms/step - accuracy: 0.6996 - loss: 0.7193 - val_accuracy: 0.5118 - val_loss: 1.5299
Epoch 3/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 21s 186ms/step - accuracy: 0.7977 - loss: 0.4912 - val_accuracy: 0.5231 - val_loss: 1.8068
Epoch 4/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 20s 176ms/step - accuracy: 0.8786 - loss: 0.3199 - val_accuracy: 0.5308 - val_loss: 1.6851
Epoch 5/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 20s 177ms/step - accuracy: 0.9124 - loss: 0.2409 - val_accuracy: 0.5036 - val_loss: 2.2362
Epoch 6/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 21s 186ms/step - accuracy: 0.9348 - loss: 0.1861 - val_accuracy: 0.5166 - val_loss: 2.4746
Epoch 7/10
106/106 ━━━━━━━━━━━━━━━━━━━━ 22s 198ms/step - accuracy: 0.9585 - loss: 0.1264 - val_accuracy: 0.4911 - val_loss: 3.0490

In [10]:
import tkinter as tk
from tkinter import filedialog, messagebox
from tkinter import ttk

# Predict image and load treatment
def predict_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(64, 64))
    img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    
    # Make a prediction
    prediction = model.predict(img_array)
    class_index = np.argmax(prediction)
    
    # Map prediction to stage using exact folder names
    classes = ['Non Demented', 'Very Mild Dementia', 'Mild Dementia', 'Moderate Dementia']
    predicted_stage = classes[class_index]
    
    # Load treatment based on classification
    treatment = load_treatment(predicted_stage)
    
    # Return predicted stage and probabilities
    return predicted_stage, prediction[0]

# Update the GUI with prediction results
def update_prediction():
    if not image_path.get():
        messagebox.showwarning("Warning", "Please select an image first.")
        return

    predicted_stage, probabilities = predict_image(image_path.get())
    
    # Update label with predicted stage and treatment
    prediction_label.config(text=f"Predicted Stage: {predicted_stage}", font=("Arial", 14, "bold"))
    treatment_label.config(text=f"Recommended Treatment: \n{load_treatment(predicted_stage)}", font=("Arial", 12))
    
    # Update progress bars
    for i, class_name in enumerate(['Non Demented', 'Very Mild Dementia', 'Mild Dementia', 'Moderate Dementia']):
        progress_var[i].set(probabilities[i])
        progress_bar[i].config(maximum=1, value=probabilities[i])
        progress_label[i].config(text=f"{class_name}: {probabilities[i]:.2f}", font=("Arial", 10))

# Open file dialog to select image
def select_image():
    file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])
    if file_path:
        image_path.set(file_path)

# Clear the input and reset the GUI
def clear_input():
    image_path.set("")
    prediction_label.config(text="")
    treatment_label.config(text="")
    for i in range(4):
        progress_var[i].set(0)
        progress_bar[i].config(value=0)
        progress_label[i].config(text="")

# Initialize the GUI
root = tk.Tk()
root.title("Brain Image Classifier")
root.geometry("500x600")
root.configure(bg='#f0f0f5')

# Variable to hold image path
image_path = tk.StringVar()

# Create UI elements
header_label = tk.Label(root, text="Brain Image Classifier", font=("Arial", 18, "bold"), bg='#f0f0f5')
header_label.pack(pady=10)

tk.Label(root, text="Select an Image to Classify:", font=("Arial", 12), bg='#f0f0f5').pack(pady=10)
image_entry = tk.Entry(root, textvariable=image_path, width=50, font=("Arial", 10))
image_entry.pack(pady=5)

select_button = tk.Button(root, text="Select Image", command=select_image, bg="#4CAF50", fg="white", font=("Arial", 12))
select_button.pack(pady=5)

predict_button = tk.Button(root, text="Predict", command=update_prediction, bg="#2196F3", fg="white", font=("Arial", 12))
predict_button.pack(pady=5)

clear_button = tk.Button(root, text="Clear", command=clear_input, bg="#f44336", fg="white", font=("Arial", 12))
clear_button.pack(pady=5)

prediction_label = tk.Label(root, text="", font=("Arial", 12), bg='#f0f0f5')
prediction_label.pack(pady=10)

treatment_label = tk.Label(root, text="", font=("Arial", 12), bg='#f0f0f5')
treatment_label.pack(pady=10)

# Progress bars for each class
progress_var = [tk.DoubleVar() for _ in range(4)]
progress_bar = []
progress_label = []

progress_frame = ttk.Frame(root)
progress_frame.pack(pady=20, fill=tk.X, padx=10)

tk.Label(progress_frame, text="Classification Probabilities:", font=("Arial", 14, "bold")).pack(anchor='w')

for i, class_name in enumerate(['Non Demented', 'Very Mild Dementia', 'Mild Dementia', 'Moderate Dementia']):
    frame = ttk.Frame(progress_frame)
    frame.pack(pady=5, fill=tk.X)

    bar = ttk.Progressbar(frame, variable=progress_var[i], maximum=1, length=300)
    bar.pack(side=tk.LEFT, fill=tk.X, expand=True, padx=5)

    label = tk.Label(frame, text="", font=("Arial", 10))
    label.pack(side=tk.LEFT, padx=10)
    progress_bar.append(bar)
    progress_label.append(label)

root.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step
